# Read Data

## Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random
import numpy as np
from string import ascii_lowercase
from string import ascii_uppercase
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from category_encoders import *
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler   # standardize the data
from sklearn.decomposition import PCA  # doing PCA 
from scipy.stats import norm

## Load Data

In [9]:
dat = pd.read_csv('F:/Lab/Majo Data/SVM-data.csv', header = 0)

In [10]:
dat.head()

,HPI,Extravasation,Region,Area,Volume,Sphericity,Oblate,Prolate,Ratio
0,24,1,CP,1609.74,3003.64,0.625406,0.149171,0.449330,0.535930
1,24,0,CV,1881.01,5457.05,0.796890,0.224582,0.790789,0.344694
2,24,1,CP,7942.66,26543.00,0.541774,0.342604,0.384072,0.299237
3,24,0,CV,4023.96,14014.30,0.698573,0.392206,0.464822,0.287132
4,24,1,CV,1936.66,6827.60,0.898687,0.348174,0.394179,0.283652


#  Preprocessing

In [11]:
# delete NAN value
data = dat.copy()
data = data.dropna()

In [12]:
# Seperating the features and target
y = data.Extravasation
x_numeric = data.drop(['Extravasation', 'HPI', 'Region', 'Ratio', 'Prolate'],axis=1)

### PCA for numerical variables

#### Standardize data

In [13]:
# Standardizing the features
x_numeric = StandardScaler().fit_transform(x_numeric)

PCA component includes two (not use two components)

In [107]:
# Doing PCA
#pca = PCA(n_components=2)
#principalComponents = pca.fit_transform(x_numeric)
#principalDf = pd.DataFrame(data = principalComponents
#             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
#maptfyn = {1 : 'T', 0 : 'F'}
#y = y.map(maptfyn)
#y

In [ ]:
# Visualization 2D projection
#fig = plt.figure(figsize = (8,8))
#ax = fig.add_subplot(1,1,1) 
#ax.set_xlabel('Principal Component 1', fontsize = 15)
#ax.set_ylabel('Principal Component 2', fontsize = 15)
#ax.set_title('2 component PCA', fontsize = 20)
#targets = ['T', 'F']
#colors = ['r', 'g']
#for y, color in zip(targets,colors):
 #   indicesToKeep = finalDf['Extravasation'] == y
 #   ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
  #             , finalDf.loc[indicesToKeep, 'principal component 2']
  #            , c = color
  #             , s = 50)
#ax.legend(targets)
#ax.grid()

#### Keep One Component from PCA

In [14]:
# Doing PCA on x_numeric part, only reserve one main component
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x_numeric)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1'])

In [15]:
finalDf = pd.concat([principalDf, y], axis = 1)

In [16]:
finalDf

,principal component 1,Extravasation
0,-0.316914,1.0
1,-0.906121,0.0
2,-0.204281,1.0
3,-0.860259,0.0
4,-1.401507,1.0
...,...,...
216,1.354876,1.0
217,-1.112084,1.0
218,-1.031106,1.0
219,1.153960,0.0


In [104]:
#finalDf = finalDf.dropna()

### Combine all independent features

In [17]:
Df = pd.concat([finalDf, data[['HPI']], data[['Region']]], axis = 1)

In [18]:
Df

,principal component 1,Extravasation,HPI,Region
0,-0.316914,1.0,24.0,CP
1,-0.906121,0.0,24.0,CV
2,-0.204281,1.0,24.0,CP
3,-0.860259,0.0,24.0,CV
4,-1.401507,1.0,24.0,CV
...,...,...,...,...
216,1.354876,1.0,96.0,CP
217,-1.112084,1.0,96.0,CV
218,-1.031106,1.0,96.0,CP
219,1.153960,0.0,96.0,ISV


In [15]:
# Encoding
#maptfyn02 = {'T':1, 'F':0}
#Df['Extravasation'] = Df['Extravasation'].map(maptfyn02)

### Split variables, train data and test data

In [19]:
Df = Df.dropna()

In [20]:
y_new = Df['Extravasation']

In [21]:
x_new = Df.drop(['Extravasation'], axis = 1)

In [22]:
# Split x and y into training part and testing part, the partition is 8：2
x_train,x_test,y_train,y_test=train_test_split(x_new,y_new,test_size=0.2)

### Encoding

In [23]:
def ord_encode(X):
    x = X.copy()
    map1 = {24 : 0, 72 : 1, 96 : 2}
    x['HPI'] = x['HPI'].map(map1)
    return x

x_train = ord_encode(x_train)
x_test = ord_encode(x_test)

In [24]:
# Since the Region number > 4, consider using target encoding 
enc = TargetEncoder(cols=['Region']).fit(x_train, y_train)
x_train = enc.transform(x_train)
x_test = enc.transform(x_test)

In [25]:
x_train

,principal component 1,HPI,Region
170,-1.760520,2,0.277778
134,-0.947482,1,0.277778
63,-1.902101,1,0.714285
188,-0.920652,2,0.466667
35,-1.316355,0,1.000000
...,...,...,...
62,-1.421957,0,1.000000
41,-0.097059,0,1.000000
80,2.685672,1,0.277778
191,-1.378252,2,1.000000


## Logistic Regression

In [26]:
# training model
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred_train = lr.predict(x_train)
#accurate probability for train data
p_train = np.mean(y_pred_train == y_train)
y_pred = lr.predict(x_test)
p = np.mean(y_pred == y_test)
print(p_train)
# accurate probability for test data
print(p)

0.7942857142857143
0.75


In [27]:
# Coefficients for the model
lr.coef_

array([[0.31008597, 0.69246505, 3.63669629]])

In [28]:
# intercept for the model
lr.intercept_

array([-2.40719085])

In [31]:
# logit_pvalue funtion

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

In [30]:
logit_pvalue(lr, x_train)

array([3.58125209e-06, 1.21556280e-02, 3.77348271e-03, 6.57849333e-07])